<a href="https://colab.research.google.com/github/Pavlos01232/Match_Outcome_Prediction/blob/main/Game_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch as t
import torch.nn as n
import pandas as pd
import os

print("beep boop")
print("Aston Villa loses")
print("=============================================================")
#GITHUB LOCATION:
#https://github.com/Pavlos01232/Match_Outcome_Prediction
#
#
#
#
#
#
#




#pd.read_csv('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0304.csv?raw=true')


#"DEEP learning" just means "hidden" layers



#read function
#df = pd.read_csv('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL0405.csv?raw=true',sep='\t', lineterminator='\r')
#print(df)

first = "https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/PL"
last = ".csv?raw=true"


#3 to 23
#the first two files in the training data are formatted incorrectly
#convert csv to dataframe
i=5
for i in range(5, 6):
  result = first + str('{:02.0f}'.format(i)) + str('{:02.0f}'.format(i+1)) + last
  #print(result)
  df[i] = pd.read_csv(result, sep='\t', encoding = 'unicode_escape', lineterminator='\r')
  #tensor = t.tensor(df.values)
  #print(tensor)
  #df_to_tensor(pd.read_csv(result, sep='\t', encoding = 'unicode_escape', lineterminator='\r'))

print (df)
df.to_csv(r'C:\Users\Pavlos\Desktop\export_dataframe.csv', sep='\t', encoding='utf-8')
#print (df[2])




#file_list = os.listdir('https://raw.githubusercontent.com/Pavlos01232/Match_Outcome_Prediction/main/TRAINING%20DATA/')





print("=============================================================")

i=5

stats=t.tensor(i)

#stats[0]=5
#stats[1]=2
#stats[2]=8
#stats[3]=4
#stats[4]=8
#stats[5]=3

print(stats)

t.cuda.memory_allocated(0)/1024**3

beep boop
Aston Villa loses
    Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,GBH,GBD,GBA,IWH,IWD,IWA,LBH,LBD,LBA,SBH,SBD,SBA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA  \
0    \nE0,13/08/05,Aston Villa,Bolton,2,2,D,2,2,D,M...                                                                                                                                                                                                                                                                                                     
1    \nE0,13/08/05,Everton,Man United,0,2,A,0,1,A,G...                                                                                                                                                                                                                              

0.0